In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

#함수 선언

#공기 점성도 계산
def Eta_Calculate(T_celsius):
    eta_0 = 1.716e-5
    T_0 = 273.15
    S=110.4

    T= T_celsius + 273.15 #절대 온도 변환
    # 서덜랜드 공식
    eta = eta_0*(T/T_0)**(1.5)*(T_0+S)/(T+S)

    return eta 

#레이놀즈 수 계산
def Calculate_Reynolds(U, r, eta, P_a=1.2):

    if eta ==0:
        return 0
    
    Re = (2*P_a*r*U)/eta

    return Re

#항력 계수 계산
def Drag_coefficient_calculate(Re):

    if 100<Re<1000:
        cd = (98.33/Re) - (2778/(Re**2))+0.3644
        return cd
    
    elif 1000<Re<5000:
        cd =(148.62/Re) - (4.75e4/(Re**2)) +0.357
        return cd 
    
    else:
        return 0.44

#이론식 계산
def Theoretical_formula(U, r, cd, P_a=1.2):
    if U==0: return 0
    numerator = 3*P_a*cd*(U**2)
    denominator = 8*1000*9.8*r
    return numerator/denominator


#비대칭 성장 항 계산
def Asymmetric_growth(U,kappa, Aw):
    I_U = min(1.0, U/15.0)
    return kappa * I_U *Aw


def Simulate_icicle_trajectory(U, r_mm, T, beta, gamma, kappa, Aw, growth_time, v_speed):
    r=r_mm/1000.0
    v_speed_m = v_speed / 1000.0
    dt = 0.1
    steps = int (growth_time /dt)

    eta = Eta_Calculate(T)
    Re = Calculate_Reynolds(U, r, eta)
    cd = Drag_coefficient_calculate(Re)
    tan_theroy = Theoretical_formula(U, r, cd)
    growth_bias = Asymmetric_growth(U, kappa, Aw)

    #초기 상태 설정정
    x, y = [0.0], [0.0]
    current_tan_phi = 0.0

    #시간 누적 루프
    for _ in range(steps):
        tracking = beta*(tan_theroy-current_tan_phi)*dt
        growth = gamma * growth_bias * dt
        current_tan_phi += (tracking+growth)

        #궤적 좌표 업데이트
        dy = v_speed_m*dt
        dx= dy*current_tan_phi

        x.append(x[-1]+dx)
        y.append(y[-1]-dy)

   #그래프 그리기
    # plt.figure(figsize=(7, 9))
    # plt.plot(x, y, color='#4A90E2', linewidth=4, alpha=0.8, label='Icicle Trajectory')
    # plt.fill_betweenx(y, 0, x, color='#4A90E2', alpha=0.1) # 고드름 형태 시각화
    
    # plt.axhline(0, color='black', linewidth=2) # 고드름이 시작되는 지붕
    # plt.title(f"Icicle Growth Trajectory\n(Wind Speed: {U}m/s, Temp: {T}℃)", fontsize=13)
    # plt.xlabel("Horizontal Displacement (m)")
    # plt.ylabel("Vertical Length (m)")
    
    # # 축 범위 설정 (직관적인 비교를 위해 고정)
    # plt.xlim(-0.1, 0.6)
    # plt.ylim(-1.2, 0.1)
    # plt.gca().set_aspect('equal') # 비율 유지
    # plt.grid(True, linestyle=':', alpha=0.6)
    # plt.legend()
    # plt.show()

    current = 0.0
    tan_hist = [current]
    phi_hist = [np.degrees(np.arctan(current))]

    for _ in range(steps):
        tracking = beta*(tan_theroy-current)*dt
        growth = gamma * growth_bias * dt
        current += (tracking+growth)
        tan_hist.append(current)
        phi_hist.append(np.degrees(np.arctan(current)))

    t_hist = np.arange(len(tan_hist)) * dt

   #그래프 두개
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # (왼쪽) phi(t)
    ax[0].plot(t_hist, phi_hist, linewidth=3, alpha=0.9, label='phi(t)')
    ax[0].axhline(np.degrees(np.arctan(tan_theroy)), linestyle='--', linewidth=2, label='phi_theory')
    ax[0].set_title("Bending Angle Over Time", fontsize=13)
    ax[0].set_xlabel("Time (s)")
    ax[0].set_ylabel("phi (deg)")
    ax[0].grid(True, linestyle=':', alpha=0.6)
    ax[0].legend()

    # (오른쪽) 기존 궤적 그래프 
    ax[1].plot(x, y, color='#4A90E2', linewidth=4, alpha=0.8, label='Icicle Trajectory')
    ax[1].fill_betweenx(y, 0, x, color='#4A90E2', alpha=0.1)
    ax[1].axhline(0, color='black', linewidth=2)

    ax[1].set_title(f"Icicle Growth Trajectory\n(Wind Speed: {U}m/s, Temp: {T}℃)", fontsize=13)
    ax[1].set_xlabel("Horizontal Displacement (m)")
    ax[1].set_ylabel("Vertical Length (m)")
    ax[1].set_xlim(-0.1, 0.6)
    ax[1].set_ylim(-1.2, 0.1)
    ax[1].set_aspect('equal')
    ax[1].grid(True, linestyle=':', alpha=0.6)
    ax[1].legend()

    plt.tight_layout()
    plt.show()


interact(Simulate_icicle_trajectory,
         U = FloatSlider(min=0, max=15, step=0.5, value=5, description='풍속 U(m/s)'),
         r_mm = FloatSlider(min=1.0, max=10.0, step=0.1, value=2.44, description='반지름 r(mm)'),
         T = FloatSlider(min=-20, max=0, step=1, value=-9, description='온도 T(℃)'),
         beta = FloatSlider(min=0.1, max=10, step=0.1, value=5.0, description='추적속도 β'),
         gamma = FloatSlider(min=0.1, max=5, step=0.1, value=1.5, description='누적계수 γ'),
         kappa = FloatSlider(min=0.1, max=5, step=0.1, value=2.0, description='성장스케일 κ'),
         Aw = FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='성장차이 Aw'),
         growth_time = IntSlider(min=60, max=1800, step=60, value=600, description='성장시간(s)'),
         v_speed = FloatSlider(min=1, max=10, step=0.5, value=3, description='성장속도(mm/s)'))


interactive(children=(FloatSlider(value=5.0, description='풍속 U(m/s)', max=15.0, step=0.5), FloatSlider(value=2…

<function __main__.Simulate_icicle_trajectory(U, r_mm, T, beta, gamma, kappa, Aw, growth_time, v_speed)>